In [1]:
import json
import urllib
import requests
import csv

In [156]:
import pandas as pd
import calendar

#
# Read JSON request into a pandas Dataframe
#

colNames = ['index', 'Rain', 'HiTemp', 'LowTemp', 'Year', 'Month', 'Day']
df = pd.read_json('/var/www/html/000/monthly.txt')
df = df.drop(df.columns[[0]], axis=1)

#
# Get month name from month number and make some conversions 
#

Date = (df['Day']).astype(int)
month_num  = (df['Month']).astype(int)
month_num = month_num[0]
month_name = calendar.month_name[month_num]
Year = (df['Year']).astype(int)
year = Year[0]

High = (df['HiTemp']).round(0).astype(int)
Low = (df['LowTemp']).round(0).astype(int)
Avg = ((High + Low)/2).round(0).astype(int)
HDD = (65 - Avg).round(0).astype(int)
HDD = HDD.where(HDD > 0, 0) 
CDD = (Avg - 65).round(0).astype(int)
CDD = CDD.where(CDD > 0, 0) 
Rainfall = (df['Rain']).astype(float).fillna(0)

month_High_avg = High.mean().round(1) 
month_Low_avg = Low.mean().round(1) 
month_avg = Avg.mean().round(1)
totHDD = HDD.sum()
totCDD = CDD.sum()
totRainfall = Rainfall.sum().round(2)

df.insert(6, 'Average', Avg)
df.insert(7, 'HDD', HDD)
df.insert(8, 'CDD', CDD)
df.insert(9, 'High', High)
df.insert(10, 'Low', Low)
df.insert(11, 'Rainfall', Rainfall)
df.insert(12, 'Date', Date)

df = df.reindex(columns=['Year', 'Month', 'Date', 'High', 'Low', 'Average', 'HDD', 'CDD','Rainfall'])
df = df.drop(df.columns[[0,1]], index = None, axis=1)

    Date  High  Low  Average  HDD  CDD  Rainfall
0      1    66   46       56    9    0      0.00
1      2    63   45       54   11    0      0.00
2      3    72   43       58    7    0      0.00
3      4    67   46       56    9    0      0.00
4      5    72   50       61    4    0      0.00
5      6    74   63       68    0    3      0.04
6      7    70   52       61    4    0      0.01
7      8    60   39       50   15    0      0.00
8      9    59   33       46   19    0      0.00
9     10    60   56       58    7    0      1.81
10    11    56   50       53   12    0      0.07
11    12    59   52       56    9    0      0.13
12    13    60   45       52   13    0      0.00
13    14    52   35       44   21    0      0.00
14    15    61   34       48   17    0      0.00
15    16    69   44       56    9    0      0.00
16    17    54   46       50   15    0      0.00
17    18    60   46       53   12    0      0.00
18    19    63   56       60    5    0      0.84
19    20    63   41 

In [162]:
from pretty_html_table import build_table

html_table_blue_light = build_table(df, 'blue_light', text_align='center')

with open('/var/www/html/000/monthlyTable.html', 'w') as f:
    f.write(html_table_blue_light)